In [21]:
from functions.filtrar_datos_real_vert import filtrar_datos_vertical
from functions.PitotProcess import PitotProcess
import pandas as pd
import numpy as np

In [22]:

# Ruta del archivo Excel
archivo_excel = "prueba_tunel_2025-01-15_15-57-53.xlsx"
# Leer el archivo Excel y convertirlo en un dataset
# Por defecto, lee la primera hoja del archivo
dataset = pd.read_excel(archivo_excel)

In [23]:

dataset = dataset.iloc[:,:8]#se toman solo las 8 columnas de los datos tomados
datatemp = pd.DataFrame(columns=dataset.columns)#datafrae temporal para el datahistory
datatemp = dataset.head(34)#los primeros 34 valores de la medicion se nulifican para definir el filtro

In [24]:
database_pitots = pd.DataFrame(columns=["P1","P2"]) #creacion de database para guardar datos de pitots

In [25]:
local_pressure = 29.99 #esto es la presion atmosferica las unidades son desconocidas
euler_angles = [0,0,0] #angulos de euler dados por IMU anulados por pruebas estaticas
for i in range(len(dataset)):
    local_temp = datatemp[['Temperatura_1','Temperatura_2','Temperatura_3','Temperatura_4']].mean().mean()
    new_Value = dataset.loc[i]

    filtered_data, datatemp = filtrar_datos_vertical(new_Value,datatemp)
    pressures_data = filtered_data[0:4]
    proccessed_pressure = PitotProcess(pressures_data,local_pressure,local_temp,euler_angles)
    database_pitots.loc[i] = [proccessed_pressure[0], proccessed_pressure[1]]

    #print(filtered_data-new_Value)

c:\Users\jbero\Desktop\VOLTA\CODIGO\Modelo-Matematico\simulacion\functions\filtrar_datos_real_vert.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_history.iloc[1:, i] = data_history.iloc[:-1, i].values  # Desplazar hacia abajo
c:\Users\jbero\Desktop\VOLTA\CODIGO\Modelo-Matematico\simulacion\functions\filtrar_datos_real_vert.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_history.iloc[0, i] = dato  # Insertar el nuevo dato en la primera fila
c:\Users\jbero\Desktop\VOLTA\CODIGO\Modelo-Matematico\simulacion\functions\filtrar_datos_real_vert.py:28: SettingWithCopyWarning: 
A value is trying to be set on

In [26]:
database_pitots #RESULTADO FINAL, PAREJAS DE PITOTS

,P1,P2
0,"[351.65561556676005, 613.7707032542744, 0.0]","[633.6500772228303, 608.1757663797488, 0.0]"
1,"[351.26857026315434, 612.4497603317675, 0.0]","[633.679202747716, 614.6145526712345, 0.0]"
2,"[346.4377543626862, 609.7834807359624, 0.0]","[633.7150583283781, 607.414277671617, 0.0]"
3,"[345.8449820933524, 613.3042020355672, 0.0]","[632.5328672560712, 611.397759697703, 0.0]"
4,"[349.6511391480801, 615.6521433180595, 0.0]","[632.5240822559665, 611.7271837370172, 0.0]"
...,...,...
504,"[362.7473185087098, 612.6222537974363, 0.0]","[637.4166645569268, 610.5561361018209, 0.0]"
505,"[361.8337509378001, 609.5278458843051, 0.0]","[635.5983519947181, 609.6939371417677, 0.0]"
506,"[365.51817726447, 614.1407477481797, 0.0]","[636.8541737054211, 611.8760969622003, 0.0]"
507,"[369.1118349728718, 609.9132475713426, 0.0]","[641.4233784958525, 609.4350468605503, 0.0]"
